In [12]:
! pip install scikit-learn


^C


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
    --------------------------------------- 0.3/11.1 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.1 MB 3.0 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 3.1 MB/s eta 0:00:04
   ------- -------------------------------- 2.1/11.1 MB 3.2 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.1 MB 3.3 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/11.1 MB 3.2 MB/s eta 0:00:03
   ---------------- ----------------------- 4.5/11.1 MB 3.3 MB/s eta 0:00:03
   ------------------ --------------------- 5.2/11.1 MB 3.4 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.1 MB 3.4 MB/s eta 0:00:02
   ----------------------- ---------------- 6.6/11.1 MB 3.4 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.1 MB 3.3 MB/s eta 0:00:02
   ---------------------------- ----------- 7.9/11.1 MB 3.3 MB/s eta 0:00:01
   ----------

In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [38]:
model=load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as f:
    label_encode_gender=pickle.load(f)

with open('ohe_geo.pkl','rb') as f:
    ohe_geo=pickle.load(f)

with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)

c:\Users\umang\Downloads\annclassification\venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\umang\Downloads\annclassification\venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\umang\Downloads\annclassification\venv\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from v

In [39]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [40]:
df=pd.DataFrame([input_data])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [41]:
df['Gender']=label_encode_gender.transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [42]:
ohe_geo

OneHotEncoder()

In [43]:
temp=ohe_geo.transform(df[['Geography']]).toarray()
temp_df=pd.DataFrame(temp,columns=ohe_geo.get_feature_names_out(['Geography']))
temp_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
input_df=pd.concat([df.drop('Geography',axis=1),temp_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [45]:
input_scale=scaler.transform(input_df)
input_scale

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [46]:
pred=model.predict(input_scale)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step


array([[0.03354013]], dtype=float32)

In [47]:
pred_proba=pred[0][0]
pred_proba

np.float32(0.033540133)

In [48]:
if pred_proba > 0.5:
    print('Customer is likely to churn')
else:
    print('Customer is not likely to churn')

Customer is not likely to churn
